# SmartSwitch Tutorial 04: Type-Based Dispatch

**Welcome back!** This is the fourth tutorial in the SmartSwitch series.

In this notebook you'll learn:
- ✅ How to dispatch automatically based on argument types
- ✅ How to eliminate isinstance() chains
- ✅ When type dispatch is better than `functools.singledispatch`

**Time**: ~10 minutes

**Prerequisites**: Complete Tutorials 01-03 first

---

## The Problem

You need to process different types of data differently.

**Traditional approach** - Multiple isinstance() checks:

In [ ]:
# Traditional: isinstance() chains (ugly!)
def process(data):
    if isinstance(data, str):
        return data.upper()
    elif isinstance(data, int):
        return data * 2
    elif isinstance(data, list):
        return len(data)
    elif isinstance(data, dict):
        return list(data.keys())
    else:
        return None

print(process("hello"))
print(process(42))
print(process([1, 2, 3]))
print(process({"a": 1, "b": 2}))

**Problems:**
- ❌ Logic for all types in one function
- ❌ Hard to test individual type handlers
- ❌ Difficult to add new types
- ❌ Mixed concerns in one place

## The SmartSwitch Solution

Use `typerule` to declare type-based routing:

In [ ]:
from smartswitch import Switcher

processor = Switcher()

@processor(typerule={'data': str})
def process_string(data):
    return data.upper()

@processor(typerule={'data': int})
def process_number(data):
    return data * 2

@processor(typerule={'data': list})
def process_list(data):
    return len(data)

@processor(typerule={'data': dict})
def process_dict(data):
    return list(data.keys())

# Default handler
@processor
def process_other(data):
    return None

# Automatic dispatch based on type!
print(processor()(data="hello"))     # → HELLO
print(processor()(data=42))          # → 84
print(processor()(data=[1, 2, 3]))   # → 3
print(processor()(data={"a": 1}))    # → ['a']

**How it works:**

```python
processor()  # ← Returns dispatcher
            # ↓ Call with named argument
processor()(data=42)
```

SmartSwitch:
1. Checks the **type** of each argument
2. Matches against `typerule` declarations
3. Calls the **first matching handler**

## Benefits

✅ **Separation of concerns** - Each type has its own handler

✅ **Testable** - Test each type handler independently

✅ **Extensible** - Just add another `@processor(typerule=...)`

✅ **Type-safe** - Handlers see the correct type

## Multiple Parameters

Type-check multiple arguments:

In [ ]:
from smartswitch import Switcher

calculator = Switcher()

# Both must be int
@calculator(typerule={'a': int, 'b': int})
def add_ints(a, b):
    return a + b

# Both must be str
@calculator(typerule={'a': str, 'b': str})
def concat_strings(a, b):
    return a + b

# Mixed types
@calculator(typerule={'a': str, 'b': int})
def repeat_string(a, b):
    return a * b

@calculator
def unsupported(a, b):
    return "Unsupported types"

print(calculator()(a=10, b=20))        # → 30
print(calculator()(a="hello", b=3))    # → hellohellohello
print(calculator()(a="hi", b="!"))     # → hi!
print(calculator()(a=1.5, b=2.5))      # → Unsupported types

## Union Types

Match multiple types for one parameter:

In [ ]:
from smartswitch import Switcher

formatter = Switcher()

# Match int OR float
@formatter(typerule={'value': int | float})
def format_number(value):
    return f"Number: {value:.2f}"

@formatter(typerule={'value': str})
def format_string(value):
    return f"String: '{value}'"

@formatter
def format_other(value):
    return f"Other: {value}"

print(formatter()(value=42))       # → Number: 42.00
print(formatter()(value=3.14))     # → Number: 3.14
print(formatter()(value="test"))   # → String: 'test'
print(formatter()(value=[1,2,3]))  # → Other: [1, 2, 3]

## Custom Classes

Type dispatch works with any class:

In [ ]:
from dataclasses import dataclass
from smartswitch import Switcher

@dataclass
class User:
    name: str
    email: str

@dataclass
class Product:
    title: str
    price: float

serializer = Switcher()

@serializer(typerule={'obj': User})
def serialize_user(obj):
    return {"type": "user", "name": obj.name, "email": obj.email}

@serializer(typerule={'obj': Product})
def serialize_product(obj):
    return {"type": "product", "title": obj.title, "price": obj.price}

user = User("Alice", "alice@example.com")
product = Product("Laptop", 999.99)

print(serializer()(obj=user))
print(serializer()(obj=product))

## Try It Yourself!

Build a validator that handles different input types:

In [ ]:
from smartswitch import Switcher

validator = Switcher()

@validator(typerule={'value': str})
def validate_string(value):
    return len(value) > 0 and len(value) < 100

# TODO: Add more validators!
# - int: must be positive
# - list: must not be empty
# - dict: must have 'id' key

print(validator()(value="test"))  # → True

## Combining Type and Value Rules

You can use **both** `typerule` and `valrule` together:

In [ ]:
from smartswitch import Switcher

processor = Switcher()

# Must be int AND greater than 100
@processor(typerule={'value': int}, 
           valrule=lambda value: value > 100)
def process_large_int(value):
    return f"Large int: {value}"

# Must be int (but not large)
@processor(typerule={'value': int})
def process_small_int(value):
    return f"Small int: {value}"

# Must be str AND non-empty
@processor(typerule={'value': str},
           valrule=lambda value: len(value) > 0)
def process_string(value):
    return f"String: {value}"

@processor
def process_other(value):
    return "Other"

print(processor()(value=200))    # → Large int: 200
print(processor()(value=50))     # → Small int: 50
print(processor()(value="hi"))   # → String: hi

## Real-World Example: Data Processor Pipeline

Process different data types through a pipeline:

In [ ]:
from smartswitch import Switcher
import json

data_processor = Switcher()

@data_processor(typerule={'data': dict})
def process_json_dict(data):
    # Already a dict, just validate
    return {"type": "dict", "keys": list(data.keys())}

@data_processor(typerule={'data': str})
def process_json_string(data):
    # Parse JSON string
    try:
        parsed = json.loads(data)
        return {"type": "parsed_json", "data": parsed}
    except json.JSONDecodeError:
        return {"type": "plain_string", "length": len(data)}

@data_processor(typerule={'data': bytes})
def process_binary(data):
    # Decode bytes
    return {"type": "binary", "size": len(data)}

@data_processor(typerule={'data': list})
def process_list(data):
    return {"type": "list", "count": len(data), "first": data[0] if data else None}

# Test
print(data_processor()(data={"key": "value"}))
print(data_processor()(data='{"json": true}'))
print(data_processor()(data="plain text"))
print(data_processor()(data=b"binary"))
print(data_processor()(data=[1, 2, 3]))

## SmartSwitch vs functools.singledispatch

Python's `singledispatch` only works with the **first argument's type**.

SmartSwitch supports:
- ✅ Multiple parameter type checking
- ✅ Named parameters
- ✅ Combining with value rules
- ✅ Union types

In [ ]:
from functools import singledispatch
from smartswitch import Switcher

# singledispatch: Only first arg!
@singledispatch
def process_single(obj):
    return "default"

@process_single.register
def _(obj: int):
    return f"int: {obj}"

# SmartSwitch: Multiple args!
sw = Switcher()

@sw(typerule={'a': int, 'b': str})
def process_multi(a, b):
    return f"int+str: {a}, {b}"

@sw(typerule={'a': str, 'b': int})
def process_multi2(a, b):
    return f"str+int: {a}, {b}"

print(process_single(42))
print(sw()(a=10, b="hello"))
print(sw()(a="world", b=20))

## When to Use Type-Based Dispatch

This pattern is perfect for:

✅ **Polymorphic functions** - Process different types differently

✅ **Serialization** - Convert objects to JSON/XML/etc

✅ **Validation** - Type-specific validation logic

✅ **Format conversion** - Handle multiple input formats

⚠️ **Consider alternatives for**:
- Single-parameter dispatch → use `functools.singledispatch`
- Complex inheritance → use polymorphism (OOP)

## Exercise: Build a Type Converter

Create a converter that transforms types:

In [ ]:
from smartswitch import Switcher

converter = Switcher()

@converter(typerule={'value': str})
def str_to_int(value):
    try:
        return int(value)
    except ValueError:
        return None

# TODO: Add more converters!
# - int to str
# - list to tuple
# - dict to list of tuples

print(converter()(value="42"))  # → 42

## Advanced: Generic Types

Type checking works with generic types too:

In [ ]:
from typing import List, Dict
from smartswitch import Switcher

processor = Switcher()

# Note: Runtime type checking doesn't validate List[int] contents
# It only checks that it's a list!
@processor(typerule={'items': list})
def process_list(items):
    return f"List with {len(items)} items"

@processor(typerule={'mapping': dict})
def process_dict(mapping):
    return f"Dict with keys: {list(mapping.keys())}"

print(processor()(items=[1, 2, 3]))
print(processor()(mapping={"a": 1}))

## Summary

You learned:

✅ **Type-based routing** with `typerule={'param': Type}`

✅ **Multiple parameters** - Check types of all arguments

✅ **Union types** - Match multiple types with `int | float`

✅ **Combine rules** - Use `typerule` + `valrule` together

---

## What's Next?

You've completed the core SmartSwitch tutorials! 🎉

**Optional advanced topics:**
- Tutorial 05: Logging & History (NEW in v0.4.0)
- Tutorial 06: Real-World Application

📖 **Documentation**:
- [Type Rules Guide](https://smartswitch.readthedocs.io/guide/typerules/)
- [Best Practices](https://smartswitch.readthedocs.io/guide/best-practices/)
- [API Reference](https://smartswitch.readthedocs.io/api/switcher/)

---

**Questions?** Open an issue on [GitHub](https://github.com/genropy/smartswitch/issues)